# TV Script Generation
In this project, you'll generate your own [Simpsons](https://en.wikipedia.org/wiki/The_Simpsons) TV scripts using RNNs.  You'll be using part of the [Simpsons dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data) of scripts from 27 seasons.  The Neural Network you'll build will generate a new TV script for a scene at [Moe's Tavern](https://simpsonswiki.com/wiki/Moe's_Tavern).
## Get the Data
The data is already provided for you.  You'll be using a subset of the original dataset.  It consists of only the scenes in Moe's Tavern.  This doesn't include other versions of the tavern, like "Moe's Cavern", "Flaming Moe's", "Uncle Moe's Family Feed-Bag", etc..

In [1]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper

data_dir = './data/simpsons/moes_tavern_lines.txt'
text = helper.load_data(data_dir)
# Ignore notice, since we don't use it for analysing the data
text = text[81:]

## Explore the Data
Play around with `view_sentence_range` to view different parts of the data.

In [2]:
text[:1000]

"Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.\nBart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.\nMoe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?\nMoe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.\nMoe_Szyslak: What's the matter Homer? You're not your normal effervescent self.\nHomer_Simpson: I got my problems, Moe. Give me another one.\nMoe_Szyslak: Homer, hey, you should not drink to forget your problems.\nBarney_Gumble: Yeah, you should only drink to enhance your social skills.\n\n\nMoe_Szyslak: Ah, isn't that nice. Now, there is a politician who cares.\nBarney_Gumble: If I ever vote, it'll be for him. (BELCH)\n\n\nBarney_Gumble: Hey Homer, how's your neighbor's store doing?\nHomer_Simpson: Lousy. He just sits there all day. He'd have a great job if he didn't own the pl

In [3]:
view_sentence_range = (0, 10)

"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import numpy as np

print('<Dataset Stats>')
print('Roughly the number of unique words: {}'.format(len({word: None for word in text.split()})))
scenes = text.split('\n\n\n')
print('Number of scenes: {}'.format(len(scenes)))
sentence_count_scene = [scene.count('\n') for scene in scenes]
print('Average number of sentences in each scene: {}'.format(np.average(sentence_count_scene)))

sentences = [sentence for scene in scenes for sentence in scene.split('\n')]
print('Number of lines: {}'.format(len(sentences)))
word_count_sentence = [len(sentence.split()) for sentence in sentences]
print('Average number of words in each line: {}'.format(np.average(word_count_sentence)))

print()
print('The sentences {} to {}:'.format(*view_sentence_range))
print('\n'.join(text.split('\n')[view_sentence_range[0]:view_sentence_range[1]]))

<Dataset Stats>
Roughly the number of unique words: 11492
Number of scenes: 262
Average number of sentences in each scene: 14.251908396946565
Number of lines: 3996
Average number of words in each line: 12.255755755755755

The sentences 0 to 10:
Moe_Szyslak: (INTO PHONE) Moe's Tavern. Where the elite meet to drink.
Bart_Simpson: Eh, yeah, hello, is Mike there? Last name, Rotch.
Moe_Szyslak: (INTO PHONE) Hold on, I'll check. (TO BARFLIES) Mike Rotch. Mike Rotch. Hey, has anybody seen Mike Rotch, lately?
Moe_Szyslak: (INTO PHONE) Listen you little puke. One of these days I'm gonna catch you, and I'm gonna carve my name on your back with an ice pick.
Moe_Szyslak: What's the matter Homer? You're not your normal effervescent self.
Homer_Simpson: I got my problems, Moe. Give me another one.
Moe_Szyslak: Homer, hey, you should not drink to forget your problems.
Barney_Gumble: Yeah, you should only drink to enhance your social skills.




## Implement Preprocessing Functions
The first thing to do to any dataset is preprocessing.  Implement the following preprocessing functions below:
- Lookup Table
- Tokenize Punctuation

### Lookup Table
To create a word embedding, you first need to transform the words to ids.  In this function, create two dictionaries:
- Dictionary to go from the words to an id, we'll call `vocab_to_int`
- Dictionary to go from the id to word, we'll call `int_to_vocab`

Return these dictionaries in the following tuple `(vocab_to_int, int_to_vocab)`

In [4]:
import numpy as np
import problem_unittests as tests

def create_lookup_tables(text):
    """
    Create lookup tables for vocabulary
    :param text: The text of tv scripts split into words
    :return: A tuple of dicts (vocab_to_int, int_to_vocab)
    """
    # TODO: Implement Function
    vocab = set(text)
    word2int = {word : idx for idx, word in enumerate(vocab)}
    int2word = {idx : word for idx, word in enumerate(vocab)}
    
    return word2int, int2word


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_create_lookup_tables(create_lookup_tables)

Tests Passed


### Tokenize Punctuation
We'll be splitting the script into a word array using spaces as delimiters.  However, punctuations like periods and exclamation marks make it hard for the neural network to distinguish between the word "bye" and "bye!".

Implement the function `token_lookup` to return a dict that will be used to tokenize symbols like "!" into "||Exclamation_Mark||".  Create a dictionary for the following symbols where the symbol is the key and value is the token:
- Period ( . )
- Comma ( , )
- Quotation Mark ( " )
- Semicolon ( ; )
- Exclamation mark ( ! )
- Question mark ( ? )
- Left Parentheses ( ( )
- Right Parentheses ( ) )
- Dash ( -- )
- Return ( \n )

This dictionary will be used to token the symbols and add the delimiter (space) around it.  This separates the symbols as it's own word, making it easier for the neural network to predict on the next word. Make sure you don't use a token that could be confused as a word. Instead of using the token "dash", try using something like "||dash||".

### First let's check the complete characters set of this text corpus

In [5]:
x = sorted(set(list(text)))
print('Symbol List:')
print(x, '\n')

punctuations = [c for c in x if not (c.isalpha() or c.isdigit())]
print('Punctuation List:')
print(punctuations)

Symbol List:
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '猫', '盲', '眉', '脌', '脿', '茅', '茫', '贸'] 

Punctuation List:
['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', ',', '-', '.', '/', ':', ';', '?', '_']


In [6]:
def token_lookup():
    """
    Generate a dict to turn punctuation into a token.
    :return: Tokenize dictionary where the key is the punctuation and the value is the token
    """
    # TODO: Implement Function
    punc_dict = {'.' :  '||Period||',
                 ',' :  '||Comma||',
                 '"' :  '||Quotation||',
                 ';' :  '||Semicolon||', 
                 '!' :  '||Exclamation||', 
                 '?' :  '||Question||', 
                 '(' :  '||Left||', 
                 ')' :  '||Right||',
                 '--' : '||Dash||', 
                 '\n' : '||Return||'}
    return punc_dict

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_tokenize(token_lookup)

Tests Passed


> 可以注意到这里并未要求处理语言中比较常见的冒号 `:` 以及 `'` 号，可能是考虑到这两个符号与单词结合的更为紧密吧？

## Preprocess all the data and save it
Running the code cell below will preprocess all the data and save it to file.

In [7]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Preprocess Training, Validation, and Testing Data
helper.preprocess_and_save_data(data_dir, token_lookup, create_lookup_tables)

# Check Point
This is your first checkpoint. If you ever decide to come back to this notebook or have to restart the notebook, you can start from here. The preprocessed data has been saved to disk.

In [8]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import helper
import numpy as np
import problem_unittests as tests

int_text, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()

## Build the Neural Network
You'll build the components necessary to build a RNN by implementing the following functions below:
- get_inputs
- get_init_cell
- get_embed
- build_rnn
- build_nn
- get_batches

### Check the Version of TensorFlow and Access to GPU

In [9]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from distutils.version import LooseVersion
import warnings
import tensorflow as tf

# Check TensorFlow Version
assert LooseVersion(tf.__version__) >= LooseVersion('1.0'), 'Please use TensorFlow version 1.0 or newer'
print('TensorFlow Version: {}'.format(tf.__version__))

# Check for a GPU
if not tf.test.gpu_device_name():
    warnings.warn('No GPU found. Please use a GPU to train your neural network.')
else:
    print('Default GPU Device: {}'.format(tf.test.gpu_device_name()))

TensorFlow Version: 1.2.1
Default GPU Device: /gpu:0


### Input
Implement the `get_inputs()` function to create TF Placeholders for the Neural Network.  It should create the following placeholders:
- Input text placeholder named "input" using the [TF Placeholder](https://www.tensorflow.org/api_docs/python/tf/placeholder) `name` parameter.
- Targets placeholder
- Learning Rate placeholder

Return the placeholders in the following tuple `(Input, Targets, LearningRate)`

#### 输入 Tensor 是二维的，目标 Tensor 也是二维的，具体维度的长度先不定，因此用None填充。

In [10]:
def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    # TODO: Implement Function
    inputs = tf.placeholder(dtype=tf.int32, shape=[None, None], name='input')
    targets = tf.placeholder(dtype=tf.int32, shape=[None, None], name='targets')
    lr = tf.placeholder(dtype=tf.float32, name='lr')
    return inputs, targets, lr


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_inputs(get_inputs)

Tests Passed


### Build RNN Cell and Initialize
Stack one or more [`BasicLSTMCells`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/BasicLSTMCell) in a [`MultiRNNCell`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell).
- The Rnn size should be set using `rnn_size`
- Initalize Cell State using the MultiRNNCell's [`zero_state()`](https://www.tensorflow.org/api_docs/python/tf/contrib/rnn/MultiRNNCell#zero_state) function
    - Apply the name "initial_state" to the initial state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the cell and initial state in the following tuple `(Cell, InitialState)`

#### 基本流程

- 定义单个 LSTM Cell 结构
- 在每个 LSTM Cell 后面追加 Dropout
- 将多个 LSTM Cell 串联起来
- 对所有 LSTM Cell 进行初始化

> tf.identity 的唯一作用就是对一个 Tensor 进行重命名

### 关于 Dropout

值得注意的是，在RNN中使用Dropout的效果，不会像CNN里那么万能好用，而是要根据情况选用。这里我一上来就在每个Cell后都加了Dropout，但是结果是100代训练后Loss依然在1以上。说明模型还是欠拟合状态，为了进一步提高模型的收敛速度，这时候就应该取消Dropout，直接用最简单的单层LSTM网络进行训练，实验结果证明这样可以在100代内抵达0.09的很低Loss水平。

In [11]:
num_cell = 1

def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    # TODO: Implement Function
    cell = tf.contrib.rnn.BasicLSTMCell(num_units=rnn_size)
#     cell = tf.contrib.rnn.BasicRNNCell(num_units=rnn_size)
#     cell = tf.contrib.rnn.DropoutWrapper(cell, output_keep_prob=0.8)
    cell = tf.contrib.rnn.MultiRNNCell([cell] * num_cell)
    
    # 用指定batch_size初始化LSTM Cell，并重命名
    init_state = cell.zero_state(batch_size=batch_size, dtype=tf.float32)
    init_state = tf.identity(init_state, name='initial_state')
    return cell, init_state


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_init_cell(get_init_cell)

Tests Passed


### Word Embedding
Apply embedding to `input_data` using TensorFlow.  Return the embedded sequence.

In [12]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Create embedding for <input_data>.
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    # TODO: Implement Function
    embed_init = tf.random_uniform(shape=(vocab_size, embed_dim), minval=-1, maxval=1)
    embedding = tf.Variable(initial_value=embed_init)
    return tf.nn.embedding_lookup(params=embedding, ids=input_data)


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_embed(get_embed)

Tests Passed


### Build RNN
You created a RNN Cell in the `get_init_cell()` function.  Time to use the cell to create a RNN.
- Build the RNN using the [`tf.nn.dynamic_rnn()`](https://www.tensorflow.org/api_docs/python/tf/nn/dynamic_rnn)
 - Apply the name "final_state" to the final state using [`tf.identity()`](https://www.tensorflow.org/api_docs/python/tf/identity)

Return the outputs and final_state state in the following tuple `(Outputs, FinalState)` 

In [13]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    # TODO: Implement Function
    outputs, finalState = tf.nn.dynamic_rnn(cell=cell, inputs=inputs, dtype=tf.float32)
    finalState = tf.identity(finalState, name='final_state')
    return outputs, finalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_rnn(build_rnn)

Tests Passed


### Build the Neural Network
Apply the functions you implemented above to:
- Apply embedding to `input_data` using your `get_embed(input_data, vocab_size, embed_dim)` function.
- Build RNN using `cell` and your `build_rnn(cell, inputs)` function.
- Apply a fully connected layer with a linear activation and `vocab_size` as the number of outputs.

Return the logits and final state in the following tuple (Logits, FinalState) 

In [14]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part of the neural network
    :param cell: RNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size: Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    # TODO: Implement Function
    embed = get_embed(input_data, vocab_size, embed_dim)
    outputs, finalState = build_rnn(cell, embed)
    out = tf.contrib.layers.fully_connected(inputs=outputs, num_outputs=vocab_size, activation_fn=None)
    return out, finalState


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_build_nn(build_nn)

Tests Passed


### Batches
Implement `get_batches` to create batches of input and targets using `int_text`.  The batches should be a Numpy array with the shape `(number of batches, 2, batch size, sequence length)`. Each batch contains two elements:
- The first element is a single batch of **input** with the shape `[batch size, sequence length]`
- The second element is a single batch of **targets** with the shape `[batch size, sequence length]`

If you can't fill the last batch with enough data, drop the last batch.

For exmple, `get_batches([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15], 2, 3)` would return a Numpy array of the following:
```
[
  # First Batch
  [
    # Batch of Input
    [[ 1  2  3], [ 7  8  9]],
    # Batch of targets
    [[ 2  3  4], [ 8  9 10]]
  ],
 
  # Second Batch
  [
    # Batch of Input
    [[ 4  5  6], [10 11 12]],
    # Batch of targets
    [[ 5  6  7], [11 12 13]]
  ]
]
```

In [15]:
def print_batch(a):
    print('Shape:', a.shape, '\n')
    con = np.concatenate(a, axis=2)
    print(con)

#### 写法1：矩阵解法

In [16]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    # TODO: Implement Function

    n_batches = len(int_text) // (batch_size * seq_length)
    lens = n_batches * batch_size * seq_length
    inputs_X = np.array(int_text[:lens]).reshape((batch_size, -1))
    inputs_Y = np.roll(int_text, -1)[:lens].reshape((batch_size, -1))
    res = np.zeros((n_batches, 2, batch_size, seq_length), dtype=np.int32)
    
    # 每个batch内分别存储输入和期望输出值对象
    for i in range(n_batches):
        res[i, 0] = inputs_X[:, i * seq_length : (i + 1) * seq_length]
        res[i, 1] = inputs_Y[:, i * seq_length : (i + 1) * seq_length]
    return res
    

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_batches(get_batches)
print_batch(get_batches(np.arange(1, 25), 3, 4))

Tests Passed
Shape: (2, 2, 3, 4) 

[[[ 1  2  3  4  5  6  7  8]
  [ 9 10 11 12 13 14 15 16]
  [17 18 19 20 21 22 23 24]]

 [[ 2  3  4  5  6  7  8  9]
  [10 11 12 13 14 15 16 17]
  [18 19 20 21 22 23 24  1]]]


#### 写法2：不使用矩阵，直接用双循环索引

In [17]:
def get_batches2(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    n_batches = len(int_text) // (batch_size * seq_length)
    result = []
    for i in range(n_batches):
        X = []
        Y = []
        for j in range(batch_size):
            idx = i * seq_length + j * seq_length
            X.append(int_text[idx : (idx + seq_length)])
            Y.append(int_text[(idx + 1) : (idx + seq_length + 1)])
        result.append([X, Y])
    return np.array(result)

tests.test_get_batches(get_batches)
print_batch(get_batches2(np.arange(1, 25), 3, 4))

Tests Passed
Shape: (2, 2, 3, 4) 

[[[ 1  2  3  4  5  6  7  8]
  [ 5  6  7  8  9 10 11 12]
  [ 9 10 11 12 13 14 15 16]]

 [[ 2  3  4  5  6  7  8  9]
  [ 6  7  8  9 10 11 12 13]
  [10 11 12 13 14 15 16 17]]]


### 使用高度重叠的样本Batch进行训练

- 感谢评审的意见

> 在你的第1个尝试里，batch是相互独立的。第2个尝试里，存在训练集复用问题。在新batch中重新使用了之前的训练集。同时第2种写法也因为训练集数据复用而丢掉部分训练集数据，从你的打印对比结果也可以看出，第2个尝试的打印结果里，丢掉了后面的数据，然后只对留下的部分进行训练，数据量减少了而且还重复训练了一次，导致的结果是拟合效果会更好，但泛化能力会很差。

- 针对 `np.arange(1, 25), batch_size=3, seq_length=4` 情况下，对比两种get_batch方法的结果

$$
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]
$$

#### Solution 1


```
     batch 1            batch 2
[[[ 1  2  3  4    |    5  6  7  8]
  [ 9 10 11 12    |   13 14 15 16]
  [17 18 19 20    |   21 22 23 24]]

 [[ 2  3  4  5    |    6  7  8  9]
  [10 11 12 13    |   14 15 16 17]
  [18 19 20 21    |   22 23 24  1]]]
```

#### Solution 2

```
     batch 1            batch 2
[[[ 1  2  3  4     |    5  6  7  8]
  [ 5  6  7  8     |    9 10 11 12]
  [ 9 10 11 12     |   13 14 15 16]]

 [[ 2  3  4  5     |    6  7  8  9]
  [ 6  7  8  9     |   10 11 12 13]
  [10 11 12 13     |   14 15 16 17]]]
```


## Neural Network Training
### Hyperparameters
Tune the following parameters:

- Set `num_epochs` to the number of epochs.
- Set `batch_size` to the batch size.
- Set `rnn_size` to the size of the RNNs.
- Set `embed_dim` to the size of the embedding.
- Set `seq_length` to the length of sequence.
- Set `learning_rate` to the learning rate.
- Set `show_every_n_batches` to the number of batches the neural network should print progress.

In [52]:
num_epochs = 100

batch_size = 128

rnn_size = 256

embed_dim = 256

num_cell = 1

seq_length = 32

learning_rate = 0.01

show_every_n_batches = 100

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
save_dir = './save'

### Build the Graph
Build the graph using the neural network you implemented.

In [53]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
from tensorflow.contrib import seq2seq

train_graph = tf.Graph()
with train_graph.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(
        logits,
        targets,
        tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Train
Train the neural network on the preprocessed data.  If you have a hard time getting a good loss, check the [forms](https://discussions.udacity.com/) to see if anyone is having the same problem.

In [54]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=train_graph) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})

        for batch_i, (x, y) in enumerate(batches):
            feed = {
                input_text: x,
                targets: y,
                initial_state: state,
                lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)

            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch {:>3} Batch {:>4}/{}   train_loss = {:.3f}'.format(
                    epoch_i,
                    batch_i,
                    len(batches),
                    train_loss))

    # Save Model
    saver = tf.train.Saver()
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch   0 Batch    0/16   train_loss = 8.821
Epoch   6 Batch    4/16   train_loss = 3.471
Epoch  12 Batch    8/16   train_loss = 2.272
Epoch  18 Batch   12/16   train_loss = 1.775
Epoch  25 Batch    0/16   train_loss = 1.136
Epoch  31 Batch    4/16   train_loss = 0.799
Epoch  37 Batch    8/16   train_loss = 0.552
Epoch  43 Batch   12/16   train_loss = 0.383
Epoch  50 Batch    0/16   train_loss = 0.282
Epoch  56 Batch    4/16   train_loss = 0.238
Epoch  62 Batch    8/16   train_loss = 0.220
Epoch  68 Batch   12/16   train_loss = 0.194
Epoch  75 Batch    0/16   train_loss = 0.137
Epoch  81 Batch    4/16   train_loss = 0.101
Epoch  87 Batch    8/16   train_loss = 0.089
Epoch  93 Batch   12/16   train_loss = 0.095
Model Trained and Saved


#### 模型在100代训练后的Training Loss收敛到0.09。使用的超参列表如下：

    - num_epochs = 100
    - batch_size = 128
    - rnn_size = 256
    - embed_dim = 256
    - num_cell = 1
    - seq_length = 32
    - learning_rate = 0.01
    - show_every_n_batches = 100

## 超参数对模型能力的影响分析


### 0. Vanilla RNN Cell vs LSTM Cell

- Vanilla RNN Cell 的结构简单，随着时间的推移，越早的输出将会被越快遗忘，即被后来的输入冲淡了（Washout），由于这种限制的存在，Vanilla RNN 的网络只能学会较短序列之间的关系，而无法学习到比较长序列之间的关系（例如引号开闭之间的序列长度经常可以达到10-20个字符），表现在文本生成上，看到的现象就是 Training Loss 很大，生成的文本语法错误众多。


- 使用单个Cell，分别测试 `BasicRNNCell` 构成的网络与 `BasicLSTMCell` 构成的网络，对比经过100代训练后的 Training Loss 如下

`Cell Type` | 100 代 `Training Loss`
--- | --- 
`BasicRNNCell`   | 3.00
`BasicLSTMCell`  | 0.09

> 通过对比表可以发现，LSTM Cell 的性能要远远好过 Vanilla Cell。


### 1. Cell 具有的记忆长度（`rnn_size`）

- 从类比的角度上看，`RNN Cell` 之于 RNN，就像 `卷积层` 之于 CNN 一样。


- 每个 `RNN Cell` 并不真的只是一个 `Cell`，而是代表整个一层 Cell，每个 Cell 都是在不同时刻的神经元，即 **“在时间维度上”** 展开的Cell。


- `rnn_size` 指的就是在时间维度上展开的具体长度值，有些类似于CNN中每个卷积层中具有的**卷积核数量**，它客观决定了模型的学习能力。例如 `BasicLSTMCell(num_units=256)` 定义的就是一个具有256个神经元的RNN隐层。


`rnn_size` | 100 代 `Training Loss`
--- | ---
64  | 2.00
128 | 0.70
256 | 0.10
512 | 0.09

> 通过对比表可以发现，增大 `rnn_size` 可以显著的提高模型的学习能力，在相同训练代数达到更低的 Loss。



### 2. Cell 级联个数（`num_cell`）

- 由于每个 RNN Cell 实际上都是一层，因此如果将多个 RNN Cell 级联，得到的网络就是具有多个隐层的RNN网络。


- `tf.contrib.rnn.MultiRNNCell` 做的是将多个 `LSTM Cell` （即多个 `LSTM Layer`）级联在一起。


`num_cell` | 100 代 `Training Loss`
--- | ---
1   | 0.09
3   | 1.00
6   | 2.00

> 通过对比表可以发现，盲目的增加网络深度会使得网络更难以被训练，在相同代数下收敛速度越来越慢。


### 3. 词嵌入维度（`embed_dim`）

`embed_dim` | 100 代 `Training Loss`
--- | ---
64  | 0.20
128 | 0.10
256 | 0.09
512 | 0.09

> 通过对比表可以发现，增加词嵌入的维度对降低模型Loss有一定影响，考虑到剧本语料的用词有一万多个，词嵌入的维度空间越大，越能分离相关词汇。


### 4. 序列长度（`seq_length`）

- `seq_length` 参数应该与语料本身的平均句子长度类似。

`seq_length` | `Training Loss` | `Epochs`
--- | --- | ---
4   | 1.825 | 12
8   | 0.878 | 25
16  | 0.283 | 50
32  | 0.090 | 100
64  | 0.030 | 200
128 | 0.016 | 400 
256 | 0.008 | 800

> 实际测试的结果显示，`seq_length` 太小时，模型将无法生成复杂的句子，只会重复一句话。不过，当 `seq_length` 不断增大时，模型生成的文本从语法角度看上去并没有什么太大区别，很难衡量到底是变得更好还是更坏了。


## 附：生成文本对比

### Seq_length = 4

```
moe_szyslak: picture, i wish i knew he'd slip up.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
homer_simpson:(to lisa) hi, this is guy husband to prison.
```

### Seq_length = 8

```
moe_szyslak: picture, i was gonna see a fictional tv.
moe_szyslak:(incredulous) whaddaya been an already rewound.
barney_gumble: come on, moe. i'm gonna get out on there.


homer_simpson:(loud sotto) thanks, moe.
moe_szyslak: well, i guess you'll have an idea what direction mecca's?
lloyd:(closes big) ah, ha, i'm so happy you can pour me.
barney_gumble: i can't go to get you back on your feet.
moe_szyslak: yeah, but you'd be honest with you, drive?(small sob)
moe_szyslak: hey, what's this?
homer_simpson: i don't know. i have a file.


homer_simpson:(sips it) nobody loves you...
moe_szyslak: yeah, i figured out that right?
carl_carlson: not to mention to think you, use my wife.
moe_szyslak:(tense) hey, there are right-- and that will make it happen to me.
carl_carlson: aw, marriage is gonna stay out out who is 'em gonna up the secret vigilante handshake. now we need to focus here. you know what i'm playing for ya?
lenny_leonard:(nods) mm hmm.
homer_simpson:(drunk) yeah, that's it.
moe_szyslak: hey, homer... and i can't not vote, it'll so homer looking at the house.
homer_simpson:(mouth full) hey, i wonder how much air let him be.
homer_simpson:(noticing band) moe? hey, would you like to buy some dame, you act like me?
moe_szyslak: well, i have no idea what-- i gotta take my medicine call?
moe_szyslak:(excited) there ya go.
```

### Seq_length = 16

```
moe_szyslak: picture, i think we've been polite long enough here.
lenny_leonard: that's not funny, homer.


teenage_bart:(aghast) well, that's halfway there.
snake_jailbird: thanks, homer, don't worry about the mr. x thing.(sitting down)
moe_szyslak: hey, hey, hey, hey! that plank's only for the night with edna, i'm gonna have to break your the secret ingredient!
seymour_skinner:(small laugh) the results came, but i am so late!
barney_gumble: oh my god! he's killed my dad.
homer_simpson: i hope he tells us to come clean.
lenny_leonard: oh, come on, there's sexy bald like... uh... what?(quickly points)
homer_simpson: witty, what he way. but i did publish five modestly successful children's books.
bart_simpson: all we need your advice.
moe_szyslak: it's an american tradition to cut people down to size up.
lenny_leonard: look at me! i can fl--(anguished noise)


moe_szyslak: what'sa matter, homer? usually you know it's like you're driving years hey hey, and be nice to me.
moe_szyslak:(explaining) part of urine!
moe_szyslak: yeah, that broad is gone, so lenny he's in the multi-purpose room.
homer_simpson:(short moan) can't someone else do it?
moe_szyslak:(on a huge sledge-hammer) we need for a while.
linda_ronstadt:(singing) when the snow starts a fallin' in moe.(big)
lisa_simpson:" i'd say man who shouldn't have you?
homer_simpson: i did. of course, it was kinda hard to think of reasons for a couple.
moe_szyslak: are you kiddin' me? you're the best thing to get along the new world?
moe_szyslak: that, you wanna try my new vance connor-politan? like vance fat, don't you do no accident.
moe_szyslak: look, i'd like ya.
```

### Seq_length = 32

```
moe_szyslak: picture that's very good at that.
homer_simpson:(gasp of awe) dressing?
moe_szyslak: their choice.
homer_simpson: you really think i was gonna use the housework homer and the entire steel mill was gay.
moe_szyslak: and what was lost in the service.
homer_simpson:(quietly) hi, moe. where's the moments in a place like?
moe_szyslak: well, i guess we are.
ned_flanders: i suppose i do take maude for granted.
barney_gumble: i don't know. the sat's are tomorrow.
moe_szyslak: ah, they say the most clothespins this?
moe_szyslak:(shocked) that's how it works? i was just drawin' wangs on the numbers from the...
moe_szyslak: i knew that in order to win her love, i want to give something back.
homer_simpson: listen, moe. i was thinking of another thing i got goin'.
the_rich_texan: gimme a party down to homer and the cars he towed!
lenny_leonard: with those jerks out of the way, i was able to park right outside!
homer_simpson:(smelling") big deal.
marge_simpson:(reading sign)" well, that helps.
bar_rag: so to recap: i had to go help?
moe_szyslak:(sobs) oh i need some coffee before i black out. homer, pass me the sugar.
homer_simpson:(to himself) this is gonna be great. now you'll have someone who'll rub your back... without being all right" the words of a heart-broken mother. for there will be no idea don't let it.
homer_simpson: i just hope we put in enough steampunk?
lenny_leonard:(mid-conversation) so who do you like, the padres or the tigers?
moe_szyslak: yeah, i always say(philosophical)" well, there's nothin' we can do about it...
homer_simpson:(chuckles) hey, people may not love homer simpson, but they love their stupid american flag.
```

### Seq_length = 64
```
moe_szyslak: picture me the--
homer_simpson:(looking in) hey, wait a minute-- this is real money!
homer_simpson:(proudly) yeah, my wife is raking it in.
fat_tony:(clears throat) greetings, homer.
homer_simpson:(happy to see him) hey, fat tony! you still with the mafia?
homer_simpson: no...(bitter) lenny, moe... i can't believe someone i never heard of is hangin' out with a guy like me.
moe_szyslak:(walking toward jukebox) a little splattered food never hurt anybody.(jubilant) now everybody shut up and dance!
moe_szyslak: yeah, that broad is some dame!
ned_flanders:(surprised) you all know edna?
barney_gumble: oh, that's sweet. i used to follow my dad to a lot of bars too.(belch)
moe_szyslak: here you go. homer is a real hero sure can do without: morning-after stationery.
moe_szyslak: and i got ya helicopter flying lessons.(to barflies) can you imagine this boozebag at the wheel of a whirlybird?
moe_szyslak:(laughs) he'd be all" look at me! i'm a tanked-up loser in private?
homer_simpson:(getting idea) why don't you just make it on and be.
barney_gumble: don't be that way.
homer_simpson: you know, i just made friends with pepto-bismol on facebook and uh... um... uh...
homer_simpson: well, i love you, of course. but a lot of things you do still drive me crazy.
homer_simpson: so you mean our whole marriage you've got to hope.
moe_szyslak: what? gimme that.
homer_simpson: my daughter. i have some fun.
bart_simpson:(jovial) are you kidding? i take it as a compliment!(chuckle) three beers, please!
barney_gumble: hey, join the party!(sickly, pathetic burp)
moe_szyslak:(on phone) ura snotball?
```

### Seq_length = 128
```
moe_szyslak: picture why i want to spend time with you. i'm makin' a stand here.
homer_simpson: the usual, moe. a beer and a wad of bills.
moe_szyslak:(sarcastic laugh) okay, ya lucky moron.
moe_szyslak: here you go, homer. a hundred and thirty-five dollars.
homer_simpson:(sagely) i used to hate the smell of your sweaty feet.(sniffs deeply) now it's the smell of victory.
moe_szyslak: aw, shut up.
moe_szyslak: hundred and ten, a hundred and twenty... and the crowds went wild.
barney_gumble: yoo hoo!
homer_simpson:(sings) so goodbye...
chief_wiggum:(sings) farewell...
apu_nahasapeemapetilon:(sings) shop kwik-e-mart and save...
all: goodbye my coney isle / goodbye my coney isle / goodbye my coney island babe!
nigel_bakerbutcher: homer, i'm a theatrical agent, and i want to spend time with you...
moe_szyslak: whoa, whoa, whoa! no offense, uh, but uh, i just ain't comfortable hanging around all night with, uh, y'know whatchamacallit... uh, swishkabobs.
waylon_smithers:(knowing) yeah, right. it's too bad. could have been fun. could have made a little money.
moe_szyslak: thank you, homer?
homer_simpson: moe, do you mind if i bring in some outside food?
homer_simpson: oh, i don't know. what would moe say about that?
homer_simpson:(slyly) but on the other hand, i put this up recently, and it's a good thing i did, 'cause it really illustrates my point.(turns back to painting) yep, good ol' swigmore u.
carl_carlson: gee, uh, when you talk about that school, your voice fills with, uh... what do you call it? human feeling.
lenny_leonard: yeah, maybe you should, uh... what's the expression? go back to go to go down and the town?
carl_carlson: ho, boy-- pigtown u. s. a.!
moe_szyslak: aw, shut up.
moe_szyslak: hundred and ten, a hundred and twenty... and the crowds went wild.
barney_gumble: yoo hoo!
homer_simpson:(sings) so goodbye...
chief_wiggum:(sings) farewell...
apu_nahasapeemapetilon:(sings) shop kwik-e-mart and save...
all: goodbye my coney isle / goodbye my coney isle / goodbye my coney island babe!
nigel_bakerbutcher: homer, i'm a theatrical agent, and i want to spend time with you...
```

### Seq_length = 256

```
moe_szyslak: picture, i'm surprised how much for the...
moe_szyslak: i knew that in order to win her love, i needed to get rid of the human garbage otherwise known as my best friends.
moe_szyslak: barney, how do you keep gettin' back in?
barney_gumble: i'm a drunk. i don't know nothin' about how i do anything.
moe_szyslak: i had to get edna out of springfield-- make a fresh start in a new town, far some day?
lenny_leonard: i am like that.
moe_szyslak: okay, gimme that. gimme it back!
homer_simpson:(morose) oh, what'll i do, moe?
moe_szyslak: well, why don't you invite him over to dinner that people out of town.
moe_szyslak: well, i guess you'll pine forever for a man who doesn't care, while i just grow old here in this bar.
waylon_smithers: here's to suffering in silence.
waylon_smithers:(intrigued noise) can't wait to see those numbers hanging from my rafters.
aged_moe:(sigh) i can't believe it's been three years since smithers left. man, have i been aging badly.(sighs) but i guess i can finally wash this forehead.
aged_moe:(smiles) divine.


troy:(tipsy) homer, i'm really touched you invited me out on the town. you're gonna be a four-star brother-in-law.
troy:(drunk but smooth) c'mere, homer.
troy: i'll let you in on a little secret...


young_homer:(cheers and:) looks like i've gotta tell ya, homer, the handwriting's on the wall. to stay afloat, this bar's gonna have to go queer.
larry: you mean it's not?
larry:(to book) wrong again," gay guide to springfield
barney_gumble: gimme a beer!
moe_szyslak:(calmly) i knew you'd be back.
moe_szyslak:(to self) santeria, you're the greatest!
homer_simpson: barney, didn't you say you were gonna stop drinking?
barney_gumble:(breaks down) i know, but it's so hard. please help me, homer.
homer_simpson: you came to the right guy. i'll straighten ya out... right after i finish this beer.
homer_simpson: ah, man that's sweet. okay, let's go.
moe_szyslak: hey, homer.(as homer sits) ah, no, no, no, don't sit there. take this seat, right next to the tap.
homer_simpson:(suspiciously) but that's barney's seat.
```

## Save Parameters
Save `seq_length` and `save_dir` for generating a new TV script.

In [55]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
# Save parameters for checkpoint
helper.save_params((seq_length, save_dir))

# Checkpoint

In [56]:
"""
DON'T MODIFY ANYTHING IN THIS CELL
"""
import tensorflow as tf
import numpy as np
import helper
import problem_unittests as tests

_, vocab_to_int, int_to_vocab, token_dict = helper.load_preprocess()
seq_length, load_dir = helper.load_params()

## Implement Generate Functions
### Get Tensors
Get tensors from `loaded_graph` using the function [`get_tensor_by_name()`](https://www.tensorflow.org/api_docs/python/tf/Graph#get_tensor_by_name).  Get the tensors using the following names:
- "input:0"
- "initial_state:0"
- "final_state:0"
- "probs:0"

Return the tensors in the following tuple `(InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)` 

In [57]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    # TODO: Implement Function
    InputTensor         = loaded_graph.get_tensor_by_name('input:0')
    InitialStateTensor  = loaded_graph.get_tensor_by_name('initial_state:0')
    FinalStateTensor    = loaded_graph.get_tensor_by_name('final_state:0')
    ProbsTensor         = loaded_graph.get_tensor_by_name('probs:0')
    return InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_get_tensors(get_tensors)

Tests Passed


### Choose Word
Implement the `pick_word()` function to select the next word using `probabilities`.

In [58]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    # TODO: Implement Function
    # 选择概率最大的单词索引，反查单词内容
    return int_to_vocab[np.argmax(probabilities)]


"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
tests.test_pick_word(pick_word)

Tests Passed


## Generate TV Script
This will generate the TV script for you.  Set `gen_length` to the length of TV script you want to generate.

In [59]:
gen_length = 500
# homer_simpson, moe_szyslak, or Barney_Gumble
prime_word = 'moe_szyslak'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(load_dir + '.meta')
    loader.restore(sess, load_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word + ':']
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
        dyn_seq_length = len(dyn_input[0])

        # Get Prediction
        probabilities, prev_state = sess.run(
            [probs, final_state],
            {input_text: dyn_input, initial_state: prev_state})
        
        # 手动降维才能运行。不知道为什么。
        probabilities = np.squeeze(probabilities)
        
        pred_word = pick_word(probabilities[dyn_seq_length - 1], int_to_vocab)

        gen_sentences.append(pred_word)
    
    # Remove tokens
    tv_script = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        tv_script = tv_script.replace(' ' + token.lower(), key)
    tv_script = tv_script.replace('\n ', '\n')
    tv_script = tv_script.replace('( ', '(')
        
    print(tv_script)

INFO:tensorflow:Restoring parameters from ./save
moe_szyslak: lisa's tavern... uh...(ad lib singing) my bar could be british / instead of arm-pittish / so why don't we all(spoken) eh, screw it.
kirk_van_houten: i could've at least stayed in shape for her.
barney_gumble:(gasps) precious alcohol! soaking into shag.
barney_gumble:(sighs) a glove slap to a little old face will / get you sat-is-fac-tion / glove slap...
carl_carlson: and the worst thing is, there's nothing in my mouth and everyone's invited.
larry: hey, your love testing?
detective_homer_simpson: now you get a karaoke machine.
grampa_simpson: that's the second, homer, you know that...(quickly points to door) get out.


moe_szyslak:(covering) uh, that's a project i'm working on... my name is harv bannister ma tonight.
carl_carlson: yeah, rumor has it marge threw him.
moe_szyslak: yeah, i can't get artie out of my head. he's like a spy in the house of moe.
homer_simpson: oh god, you're right. i really don't believe there's a c

### 生成文本分析

- 当 Training Loss 很小的时候，虽然句与句之间依然没有什么逻辑可言，但是生成的文本已经相当流畅，因为这些句子几乎都是原语料中的原话。


- 绝大多数的单词的拼写都是正确的。


- 绝大多数的标点都是正确的。

### 局限性的讨论

缺少客观评估模型性能的指标，虽然我们这里全程使用 Training Loss 判断模型的收敛情况，但是由于没有验证集，我们无从得知模型在什么时候已经进入过拟合状态。而且由于生成文本这种任务本身就没有一个合理的验证集，无法像分类任务那样可以用Validation Loss精确评估模型到底做的好到什么程度。

# The TV Script is Nonsensical
It's ok if the TV script doesn't make any sense.  We trained on less than a megabyte of text.  In order to get good results, you'll have to use a smaller vocabulary or get more data.  Luckly there's more data!  As we mentioned in the begging of this project, this is a subset of [another dataset](https://www.kaggle.com/wcukierski/the-simpsons-by-the-data).  We didn't have you train on all the data, because that would take too long.  However, you are free to train your neural network on all the data.  After you complete the project, of course.
# Submitting This Project
When submitting this project, make sure to run all the cells before saving the notebook. Save the notebook file as "dlnd_tv_script_generation.ipynb" and save it as a HTML file under "File" -> "Download as". Include the "helper.py" and "problem_unittests.py" files in your submission.